# 🚀 Hızlı Başlangıç - Model Testi

**🆓 Colab T4 Native modeller:** Qwen2-VL-2B, Phi-3, InternVL2-2B

---

In [ ]:
# Path ayarı
import sys, os
PROJECT_ROOT = os.path.dirname(os.getcwd()) if os.getcwd().endswith('notebooks') else os.getcwd()
if os.path.exists(os.path.join(PROJECT_ROOT, 'src')):
    sys.path.insert(0, PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()
    sys.path.insert(0, PROJECT_ROOT)

import torch
from PIL import Image
print(f"Project: {PROJECT_ROOT}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
from src.utils import PromptManager, print_model_table, FREE_MODELS, COLAB_MODELS
print_model_table()

In [ ]:
pm = PromptManager()
print("Promptlar:", pm.list_prompts())

## Qwen2-VL-2B Testi

In [ ]:
%%time
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor

MODEL = "Qwen/Qwen2-VL-2B-Instruct"
print(f"🔄 {MODEL} yükleniyor...")
processor = AutoProcessor.from_pretrained(MODEL)
model = Qwen2VLForConditionalGeneration.from_pretrained(MODEL, torch_dtype=torch.float16, device_map="auto")
print("✅ Yüklendi!")

In [ ]:
# Test görüntüsü
test_path = os.path.join(PROJECT_ROOT, "test_xray.png")
if not os.path.exists(test_path):
    import urllib.request
    urllib.request.urlretrieve("https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Chest_Xray_PA_3-8-2010.png/440px-Chest_Xray_PA_3-8-2010.png", test_path)

image = Image.open(test_path).convert("RGB")
prompt = pm.get_prompt("rrg", "detailed").user_prompt
print(f"📝 Prompt: {len(prompt)} chars")

In [ ]:
%%time
messages = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": prompt}]}]
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True).to("cuda")
output_ids = model.generate(**inputs, max_new_tokens=512)
output = processor.batch_decode(output_ids, skip_special_tokens=True)[0]

print("\n📋 RAPOR:")
print("="*50)
print(output)

## Prompt Karşılaştırması

In [ ]:
results = {}
for name in ["baseline", "detailed", "turkish"]:
    print(f"\n🔄 {name}...")
    p = pm.get_prompt("rrg", name).user_prompt
    msgs = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": p}]}]
    txt = processor.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    inp = processor(text=[txt], images=[image], return_tensors="pt", padding=True).to("cuda")
    out = model.generate(**inp, max_new_tokens=512)
    results[name] = processor.batch_decode(out, skip_special_tokens=True)[0]
    print(f"✅ {len(results[name])} chars")

In [ ]:
for n, r in results.items():
    print(f"\n{'='*50}\n📋 {n.upper()}\n{'='*50}")
    print(r[:300] + "..." if len(r) > 300 else r)

In [ ]:
# Temizlik
del model, processor
import gc; gc.collect(); torch.cuda.empty_cache()
print(f"🧹 GPU: {torch.cuda.memory_allocated()/1e9:.1f} GB")